In [2]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt 
from Bio import SeqIO
import pandas as pd

def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range (len(sequence)-size +1)]
def read_pseudo():
    file = SeqIO.parse('concatenated.fasta', 'fasta')
    data = [(record.id, record.seq._data) for record in file]
    seq = pd.DataFrame(data=data, columns=['id', 'sequence'])
    return seq

def read_staph():
    file = SeqIO.parse('core_gene_alignment-narsa.aln', 'fasta')
    data = [(record.id, record.seq._data) for record in file]
    seq = pd.DataFrame(data=data, columns=['id', 'sequence'])
    return seq

def pseudo_resp():
    responses= pd.read_csv('responses-pseudo.csv')
    return responses

def staph_resp():
    responses = pd.read_csv('responses-staph.csv')
    return responses

In [3]:
pseudo=read_pseudo()
sequences=pseudo['sequence']
pseudo['sequence']=pseudo.apply(lambda x: getKmers(x['sequence']),axis=1)

In [4]:
sequences.shape

(122,)

In [4]:
pseudoText=list(pseudo['sequence'])
for item in range (len(pseudoText)):
    pseudoText[item]=' '.join(pseudoText[item])


In [6]:
#takes in responses for carb 
responses=pseudo_resp()
carb=responses['carb']
carb=carb.fillna(0)
test=np.array(carb, dtype='object')
test*= 1
#changes all NA values to 0 too
new_lst = np.array(test, dtype='int64')

In [7]:
new_lst

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [8]:
#take in responses for toby 
toby= responses ['toby']
toby=toby.fillna(0)
test=np.array(toby, dtype='object')
test*=1
#changes all NA values to 0 too
new_lst2 = np.array(test, dtype='int64')

In [9]:
new_lst2

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
#tokenizing the strings into bag of words 
from sklearn.feature_extraction.text import CountVectorizer 
cv=CountVectorizer(ngram_range=(4,4))
X=cv.fit_transform(pseudoText)

In [11]:
X.shape

(122, 196510)

In [12]:
#splijtting datasets into training and testing sets for pseudomonas 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, new_lst, test_size=0.20, random_state=42)

In [13]:
#ask about grid search from Zhaoyi
from sklearn.naive_bayes import MultinomialNB
#use grid search to derive this conclusion, but I used this value for alpha for the time-being 
#alpha=additive smoothing parameter
classifier = MultinomialNB (alpha=0.1)
classifier.fit (X_train, y_train)


MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [14]:
y_pred=classifier.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("Calculation of K-mers w Naive Bayes algo for Pseudomonas")
print("Confusion Matrix \n")
print(pd.crosstab(pd.Series(y_test,name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_results(y_test, y_predicted):
    accuracy=accuracy_score (y_test, y_predicted)
    precision = precision_score (y_test, y_predicted, average='weighted')
    recall=recall_score (y_test, y_predicted, average="weighted")
    f1=f1_score (y_test, y_predicted, average="weighted")
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1=get_results(y_test, y_pred)
print("Accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall,f1 ))
#0=false
#1=true
#Confusion Matrix=True Negative, False Positive
#False Negative, True Positive
#Precision=TP/(TP+FP)=True Positive / Total predicted positive
#Recall = True Positive/Total Actual Positive
#F1=2* (Precision*Recall)/(Precision+Recall)

Calculation of K-mers w Naive Bayes algo for Pseudomonas
Confusion Matrix 

Predicted   0  1
Actual          
0          18  1
1           3  3
Accuracy = 0.840 
precision = 0.831 
recall = 0.840 
f1 = 0.828


In [16]:
#splijtting datasets into training and testing sets for pseudomonas 
X_train, X_test, y_train, y_test= train_test_split(X, new_lst2, test_size=0.20, random_state=42)
#ask about grid search from Zhaoyi
#use grid search to derive this conclusion, but I used this value for alpha for the time-being 
#alpha=additive smoothing parameter
classifier = MultinomialNB (alpha=0.1)
classifier.fit (X_train, y_train)
y_pred=classifier.predict(X_test)

print("Calculation of K-mers w Naive Bayes algo for Pseudomonas")
print("Confusion Matrix \n")
print(pd.crosstab(pd.Series(y_test,name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_results(y_test, y_predicted):
    accuracy=accuracy_score (y_test, y_predicted)
    precision = precision_score (y_test, y_predicted, average='weighted')
    recall=recall_score (y_test, y_predicted, average="weighted")
    f1=f1_score (y_test, y_predicted, average="weighted")
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1=get_results(y_test, y_pred)
print("Accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall,f1 ))


Calculation of K-mers w Naive Bayes algo for Pseudomonas
Confusion Matrix 

Predicted   0  1
Actual          
0          20  4
1           0  1
Accuracy = 0.840 
precision = 0.968 
recall = 0.840 
f1 = 0.886


In [5]:
#READINGS for staphylococcus
staph=read_staph()
sequences2=staph['sequence']
staph['sequence']=staph.apply(lambda x: getKmers(x['sequence']),axis=1)

print(sequences2.shape)

staphText=list(staph['sequence'])
for item in range (len(staphText)):
    staphText[item]=' '.join(staphText[item])


(125,)


In [6]:
#takes in responses for staph 
responses=staph_resp()
resp=responses['resp']
resp=resp.fillna(0)
test=np.array(resp, dtype='object')
test*= 1
#changes all NA values to 0 too
#new_lst3 stores responses to antibiotic test against staph
new_lst3 = np.array(test, dtype='int64')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
cv=CountVectorizer(ngram_range=(4,4))
X2=cv.fit_transform(staphText)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
#splijtting datasets into training and testing sets for pseudomonas 
X_train, X_test, y_train, y_test= train_test_split(X2, new_lst3, test_size=0.20, random_state=42)
#ask about grid search from Zhaoyi
#use grid search to derive this conclusion, but I used this value for alpha for the time-being 
#alpha=additive smoothing parameter
classifier = MultinomialNB (alpha=0.1)
classifier.fit (X_train, y_train)
y_pred=classifier.predict(X_test)

print("Calculation of K-mers w Naive Bayes algo for Staphylococcus")
print("Confusion Matrix \n")
print(pd.crosstab(pd.Series(y_test,name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_results(y_test, y_predicted):
    accuracy=accuracy_score (y_test, y_predicted)
    precision = precision_score (y_test, y_predicted, average='weighted')
    recall=recall_score (y_test, y_predicted, average="weighted")
    f1=f1_score (y_test, y_predicted, average="weighted")
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1=get_results(y_test, y_pred)
print("Accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall,f1 ))